In [6]:
import pandas as pd
from pathlib import Path
from storage.cherrytree import CherryTree
from utility.config import load_config
import sys
import redis
import dateparser

cf = load_config()
namespace = cf.get('namespace', 'document')
base_node = cf.get('base-node', None)
r = redis.Redis(decode_responses=True)


In [7]:
def load_status_data():
    ct = CherryTree(cf['index-file']) 
    di = f'{namespace}:filepath.document:index'
    data = []
    for node in [n for n in ct.nodes(base_node) if n.filepath]:
        dk = r.hget(di, node.filepath)
        if dk:
            item = {**{'name':node.name, 'level':node.level}, **r.hgetall(dk)}
            data.append(item)
        else:
            print(dk, 'not found')
    
    return pd.DataFrame(data).fillna('No Data')


In [8]:
df =load_status_data()

In [9]:
df[['name', 'date']]

,name,date
0,Cameron Assigned Story,27 September 1988
1,Muharto First Interview,28 September 1988
2,Surprise Call,7 June 1947
3,Flight to Tasik,28 September 1988
4,Third CALI Run,31 March 1947
...,...,...
121,Bambang Studies Bob,September 1948
122,Syd Dies,December 1988
123,Sabam Fires Cameron,December 1988
124,Victor told truth,December 1988
